<a href="https://colab.research.google.com/github/yaswhar/learning-genesis-sim/blob/main/Parallel%20Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install genesis-world -q
!pip install mediapy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
ICD_CONFIG_CONTENT = """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write(ICD_CONFIG_CONTENT)

In [ ]:
import genesis as gs
import numpy as np
import torch
import mediapy as media
from tqdm.notebook import tqdm

gs.init(backend=gs.cuda)

[Genesis] [18:12:29] [INFO] ╭───────────────────────────────────────────────╮
INFO:genesis:~<╭───────────────────────────────────────────────╮>~
[Genesis] [18:12:29] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
INFO:genesis:~<│┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈>~ ~~~~<Genesis>~~~~ ~<┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│>~
[Genesis] [18:12:29] [INFO] ╰───────────────────────────────────────────────╯
INFO:genesis:~<╰───────────────────────────────────────────────╯>~
[Genesis] [18:12:30] [INFO] Running on [Tesla T4] with backend gs.cuda. Device memory: 14.74 GB.
INFO:genesis:Running on ~~<[Tesla T4]>~~ with backend ~~<gs.cuda>~~. Device memory: ~~<14.74>~~ GB.
[Genesis] [18:12:31] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
INFO:genesis:🚀 Genesis initialized. 🔖 version: ~~<0.2.1>~~, 🌱 seed: ~~<None>~~, 📏 precision: '~~<32>~~', 🐛 debug: ~~<False>~~, 🎨 theme: '~~<dark>~~'.


In [ ]:
# Create Scene
scene = gs.Scene(
    show_viewer    = False,
    viewer_options = gs.options.ViewerOptions(
        camera_pos    = (3.5, -1.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
    ),
    rigid_options = gs.options.RigidOptions(
        dt                = 0.01,
    ),
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)

[Genesis] [18:12:39] [INFO] Scene <f237ad6> created.
INFO:genesis:Scene ~~~<<f237ad6>>~~~ created.


In [ ]:
# Entities
plane = scene.add_entity(
    gs.morphs.Plane(),
)

franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)

[Genesis] [18:12:49] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <8d77b79>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<0>~, uid: ~~~<<8d77b79>>~~~, morph: ~<<gs.morphs.Plane>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [18:12:49] [INFO] Preprocessing geom idx 0.
INFO:genesis:Preprocessing geom idx ~~<0>~~.
[Genesis] [18:12:49] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <d21cdaa>, morph: <gs.morphs.MJCF(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<1>~, uid: ~~~<<d21cdaa>>~~~, morph: ~<<gs.morphs.MJCF(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [18:12:50] [INFO] Preprocessing geom idx 1.
INFO:genesis:Preprocessing geom idx ~~<1>~~.
[Genesis] [18:12:56] [INFO] Preprocessing geom idx 2.


`env_spacing`: the spawned parallel envs share identical states. For visualization purpose, you can specify this parameter to ask the visualizer to distribute all the envs in a grid with a distance of (x, y) in meters between each env. Note that this only affects the visualization behavior, and doesn’t change the actual position of the entities in each env.

In [ ]:
# Build
# create 20 parallel environments
B = 10
frames = []
scene.build(n_envs=B, env_spacing=(1.0, 1.0))

[Genesis] [18:14:03] [INFO] Building scene <f237ad6>...
INFO:genesis:Building scene ~~~<<f237ad6>>~~~...
[Genesis] [18:14:17] [INFO] Compiling simulation kernels...
INFO:genesis:Compiling simulation kernels...
[Genesis] [18:15:16] [INFO] Building visualizer...
INFO:genesis:Building visualizer...


In [ ]:
cam.start_recording()
# control only 3 environments: 1, 5, and 7.
franka.control_dofs_position(
    position = torch.zeros(3, 9, device=gs.device),
    envs_idx = torch.tensor([1, 5, 7], device=gs.device),
)

for i in range(1000):
    scene.step()
    frames.append(cam.render()[0])

[Genesis] [18:16:52] [INFO] Running at 3.09 FPS (0.31 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.09>~ FPS (~<0.31>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:52] [INFO] Running at 3.24 FPS (0.32 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.24>~ FPS (~<0.32>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:52] [INFO] Running at 3.41 FPS (0.34 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.41>~ FPS (~<0.34>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:53] [INFO] Running at 3.59 FPS (0.36 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.59>~ FPS (~<0.36>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:53] [INFO] Running at 3.77 FPS (0.38 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.77>~ FPS (~<0.38>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:53] [INFO] Running at 3.97 FPS (0.40 FPS per env, 10 envs).
INFO:genesis:Running at ~<3.97>~ FPS (~<0.40>~ FPS per env, ~<10>~ envs).
[Genesis] [18:16:53] [INFO] Running at 4.18 FPS (0.42 FPS per env, 10 envs).
INFO:genesis:Runn

Since we are running simulation on GPU, in order to reduce data transfer overhead between cpu and gpu, we can use torch tensors selected using `gs.device` instead of numpy arrays (but numpy array will also work). This could bring noticeable performance gain when you need to send a tensor with a huge batch size frequently.

In [9]:
# Show Video
media.show_video(frames, fps=60)

This browser does not support the video tag.
